In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import altair as alt
import pandas as pd
import numpy as np

import pickle

from timeit import default_timer as timer

# Helper Functions

In [ ]:
def get_graph_statistics(G, mode='cell_values_only'):
    '''
    Returns a pandas dataframe of relevant statistical measurements on the graph
    Each row corresponds to one node in the graph
    '''
    print('Input graph has:', len(G.nodes()), 'nodes and', len(G.edges()), 'edges.')

    density = nx.function.density(G)
    print('Density:', density)

    # Calculate various measures on a per-node level
    if mode == 'cell_values_only':
        print('Calculating local clustering coefficient...')
        local_clustering_coefficient = nx.algorithms.cluster.clustering(G)
        print('Calculating betweeness centrality...')
        betweenness_centrality = nx.algorithms.centrality.betweenness_centrality(G)
    elif mode == 'bipartite':
        start = timer()
        print('Calculating local clustering coefficient using dot mode...')
        local_clustering_coefficient = nx.algorithms.bipartite.cluster.clustering(G)
        print('Finished calculating local clustering coefficient using dot mode')
        print('Elapsed time:', timer()-start, 'seconds\n')

        start = timer()
        cell_nodes = {n for n, d in G.nodes(data=True) if d['type']=='cell'}
        # TODO: betweeness takes a lot of time
        print('Calculating betweeness centrality...')
        betweenness_centrality = nx.algorithms.bipartite.centrality.betweenness_centrality(G, nodes=cell_nodes)
        print('Finished calculating betweeness centrality')
        print('Elapsed time:', timer()-start, 'seconds\n')

    # Construct the dataframe
    df = pd.DataFrame()
    df['node'] = local_clustering_coefficient.keys()
    df['local_clustering_coefficient'] = local_clustering_coefficient.values()
    df['betweenness_centrality'] = betweenness_centrality.values()

    return df

def get_homographs_and_identical_values(df_pairs):
    '''
    Given the pairs of instance cell nodes return a list of the global_cell values
    that are identified as homographs and as identical values

    Returns
    -------
    homographs_list: list of global cell values that are homographs

    identical_list: list of global cell values that are identical words
    '''
    df_homograph_pairs = df_pairs.loc[df_pairs['same_column'] == False]
    df_identical_pairs = df_pairs.loc[df_pairs['same_column'] == True]

    # All global cell values in `df_homograph_pairs` are homographs
    homographs_set = set(df_homograph_pairs['global_cell_val'].unique())

    # Some cell values in df_identical_pairs can still be homographs (e.g. jaguar_animal_1, jaguar_animal_2) they 
    # are identical instances but there exists a jaguar instance with a car meaning
    identical_set = set(df_identical_pairs['global_cell_val'].unique()) - homographs_set

    print('There are:', len(homographs_set), 'homograph words')
    print('There are:', len(identical_set), 'identical words')

    return list(homographs_set), list(identical_set)

def get_LCC_from_graph(G, graph_type='bipartite', mode='dot'):
    '''
    Return a dictionary of the LCC scores for each node in graph G

    Arguments
    -------
        G (networkx graph): a networkx graph to be analyzed

        graph_type (str): representation of the input graph
        must be one of {bipartite, cell_graph} 
       
    Returns
    -------
    python dictionary keyed by node name mapping to its LCC score
    '''
    print('Input graph has:', G.number_of_nodes(), 'nodes and', G.number_of_edges(), 'edges.')

    density = nx.function.density(G)
    print('Density:', density, '\n')

    # Calculate various measures on a per-node level
    if graph_type == 'cell_graph':
        start = timer()
        print('Calculating local clustering coefficient...')
        local_clustering_coefficient = nx.algorithms.cluster.clustering(G)
        print('Finished calculating local clustering coefficient \n Elapsed time:', timer()-start, 'seconds\n')
    elif graph_type == 'bipartite':
        # Find how many cell nodes only appear in one column (i.e. they have degree of 1)
        cell_nodes = {n for n, d in G.nodes(data=True) if d['type']=='cell'}
        degree_view = G.degree(cell_nodes)

        num_nodes_with_degree_greater_than_one = 0
        for node in cell_nodes:
            if degree_view[node] > 1:
                num_nodes_with_degree_greater_than_one += 1

        print('There are', num_nodes_with_degree_greater_than_one, 'cell nodes with degree greater than one. That is',\
        str(num_nodes_with_degree_greater_than_one / len(cell_nodes) * 100) + '% of all cell nodes.')

        if mode == 'dot':
            start = timer()
            print('Calculating local clustering coefficient using dot mode...')
            local_clustering_coefficient = nx.algorithms.bipartite.cluster.clustering(G, mode='dot')
            print('Finished calculating local clustering coefficient using dot mode')
            print('Elapsed time:', timer()-start, 'seconds\n')
        elif mode == 'min':
            start = timer()
            print('Calculating local clustering coefficient using min mode...')
            local_clustering_coefficient = nx.algorithms.bipartite.cluster.clustering(G, mode='min')
            print('Finished calculating local clustering coefficient using min mode')
            print('Elapsed time:', timer()-start, 'seconds\n')
        elif mode == 'max':
            start = timer()
            print('Calculating local clustering coefficient using max mode...')
            local_clustering_coefficient = nx.algorithms.bipartite.cluster.clustering(G, mode='max')
            print('Finished calculating local clustering coefficient using max mode')
            print('Elapsed time:', timer()-start, 'seconds\n')

    return local_clustering_coefficient

# Synthetic Benchmark (Bipartite Graph)

In [ ]:
synthetic_bipartite_graph = pickle.load(open('../graph_construction/combined_graphs_output/synthetic_benchmark_bipartite/bipartite/bipartite.graph', 'rb'))
LCC_dict = get_LCC_from_graph(synthetic_bipartite_graph, graph_type='bipartite', mode='dot')

In [ ]:
# Load dataframe of graph statistics from file
synthetic_bipartite_graph_stats_df = pickle.load(open('output/synthetic_example_bipartite/graph_stats_df.pickle', 'rb'))

# Assign LCC scores to each value in the dataframe
synthetic_bipartite_graph_stats_df['local_clustering_coefficient'] = np.nan
for idx in synthetic_bipartite_graph_stats_df.index:
    node = synthetic_bipartite_graph_stats_df.at[idx, 'node']
    synthetic_bipartite_graph_stats_df.at[idx, 'local_clustering_coefficient'] = LCC_dict[node]

# Remove all attribute nodes from the data frame. We only want to analyze nodes of type cell
cell_nodes = {n for n, d in synthetic_bipartite_graph.nodes(data=True) if d['type']=='cell'}
synthetic_bipartite_graph_stats_df = synthetic_bipartite_graph_stats_df.loc[synthetic_bipartite_graph_stats_df['node'].isin(cell_nodes)]
synthetic_bipartite_graph_stats_df

In [ ]:
# Filter to only include cell values with more than 1 degree. All other cell nodes cannot be homographs
cell_nodes = synthetic_bipartite_graph_stats_df['node'].values
nodes_with_degree_greater_than_1 = [n for n in cell_nodes if synthetic_bipartite_graph.degree[n] > 1]
synthetic_bipartite_graph_stats_df = synthetic_bipartite_graph_stats_df.loc[synthetic_bipartite_graph_stats_df['node'].isin(nodes_with_degree_greater_than_1)]

# Based on the ground truth label each node as homograph or unambiguous value
groundtruth_synthetic = pickle.load(open('ground_truth/synthetic_example_groundtruth_dict.pickle', 'rb'))
synthetic_bipartite_graph_stats_df['is_homograph'] = synthetic_bipartite_graph_stats_df['node'].map(groundtruth_synthetic)
synthetic_bipartite_graph_stats_df

In [ ]:
from pathlib import Path
# Create output directory for figures
Path("figures/synthetic_dataset").mkdir(parents=True, exist_ok=True)

In [ ]:
# Rename node with long name for easy plotting
synthetic_bipartite_graph_stats_df.at[3134, 'node'] = 'Coiled Anther'

topk_graph = alt.Chart(synthetic_bipartite_graph_stats_df.nlargest(55, 'betweenness_centrality'), title='').mark_bar(size=20).encode(
    x=alt.X('node:N', sort='-y', axis=alt.Axis(title='', labelAngle=-40, labelFontSize=30)),
    y=alt.Y('betweenness_centrality:Q', axis=alt.Axis(title='Betweenness Centrality', labelFontSize=28, titleFontSize=30, tickCount=8)),
    color=alt.Color('is_homograph:N', legend=alt.Legend(title='Value Type', titleLimit=0, labelLimit=500))
).properties(width=2800, height=600)
topk_graph = topk_graph.configure_axis(labelLimit=450)

topk_graph = topk_graph.configure_legend(
    labelFontSize=38,
    symbolSize=700,
    titleFontSize=35,
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top-right'
)

topk_graph.save('figures/synthetic_dataset/synthetic_bipartite_betweenness_topk_55_wide.svg')
topk_graph

## Identifying root cause of low betweenness for homograph nodes

In [ ]:
# Plot of all the homograph values and their betweeness
homographs_graph = alt.Chart(synthetic_bipartite_graph_stats_df[synthetic_bipartite_graph_stats_df['is_homograph'] == 'homograph'], title='').mark_bar(size=15).encode(
    x=alt.X('node:N', sort='-y', axis=alt.Axis(title='', labelAngle=-40, labelFontSize=23)),
    y=alt.Y('betweenness_centrality:Q', axis=alt.Axis(title='Betweenness Centrality', labelFontSize=26, titleFontSize=30, tickCount=8)),
    color=alt.Color('is_homograph:N', legend=alt.Legend(title='Value Type', titleLimit=0, labelLimit=500))
).properties(width=1600, height=800)
homographs_graph = homographs_graph.configure_axis(labelLimit=550)

homographs_graph = homographs_graph.configure_legend(
    labelFontSize=38,
    symbolSize=700,
    titleFontSize=35,
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top-right'
)

homographs_graph.save('figures/synthetic_dataset/synthetic_bipartite_betweenness_homographs.svg')
homographs_graph

In [ ]:
synthetic_bipartite_graph_stats_df[synthetic_bipartite_graph_stats_df['is_homograph'] == 'homograph']['node'].values

In [ ]:
# All homographs with low betweeness in a dataframe
homographs_with_low_betweeness = synthetic_bipartite_graph_stats_df[(synthetic_bipartite_graph_stats_df['is_homograph'] == 'homograph') &
    (synthetic_bipartite_graph_stats_df['betweenness_centrality'] < 
        synthetic_bipartite_graph_stats_df[synthetic_bipartite_graph_stats_df['node'] == 'Florida']['betweenness_centrality'].values[0])]
homographs_with_low_betweeness.sort_values(by=['betweenness_centrality'], ascending=False)

In [ ]:
print('Node Jaguar has neighbors:')
for neighbor in synthetic_bipartite_graph.neighbors('Jaguar'):
    print(neighbor, 'is connected to', len(list(synthetic_bipartite_graph.neighbors(neighbor))), 'nodes.')
print('\nNode AL has neighbors:')
for neighbor in synthetic_bipartite_graph.neighbors('AL'):
    print(neighbor, 'is connected to', len(list(synthetic_bipartite_graph.neighbors(neighbor))), 'nodes.')

It seems that homographs with low betweeness is due to a small number of cell nodes neighboring their attribute nodes. For example the node 'AL' is a homograph because it is an abbreviation for the state of "Alabama" and the country "Albania". Because the total number of countries + us_states is small the betweeness centrality is much smaller. Moreover there is considerable intersection between the two sets.

In [ ]:
homograph_df = synthetic_bipartite_graph_stats_df[synthetic_bipartite_graph_stats_df['is_homograph'] == 'homograph']
num_of_second_degree_neighbors_list = []
# Find the number of unique neighbors of neighbors for a each homograph node
for node in homograph_df['node']:
    unique_second_degree_neighbors = set()
    for neighbor in list(synthetic_bipartite_graph.neighbors(node)):
        unique_second_degree_neighbors |= set(synthetic_bipartite_graph.neighbors(neighbor))
    num_of_second_degree_neighbors_list.append(len(unique_second_degree_neighbors))
homograph_df['num_second_degree_neighbors'] = num_of_second_degree_neighbors_list
homograph_df.sort_values(by=['num_second_degree_neighbors'])

In [ ]:
points = alt.Chart(homograph_df, title='').mark_circle(size=300, clip=True).encode(
    x=alt.X('num_second_degree_neighbors:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Cardinality', labelFontSize=28, titleFontSize=30)),
    y=alt.Y('betweenness_centrality:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Betweenness Centrality', format=".1e", labelFontSize=28, titleFontSize=30, tickCount=8)),
    tooltip=['node', 'betweenness_centrality', 'num_second_degree_neighbors']
)

text = points.mark_text(
    fontSize=32,
    align='center',
    baseline='middle',
    dy=-25
).encode(
    text='node'
)

fig = (text + points).properties(width=1600, height=800)
fig.save('figures/synthetic_dataset/synthetic_bipartite_betweenness_vs_cardinality_all.svg')
fig


In [ ]:
points = alt.Chart(homograph_df, title='').mark_circle(size=300, clip=True).encode(
    x=alt.X('num_second_degree_neighbors:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Cardinality', labelFontSize=28, titleFontSize=30)),
    y=alt.Y('betweenness_centrality:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Betweenness Centrality', format=".1e", labelFontSize=28, titleFontSize=30, tickCount=8)),
    tooltip=['node', 'betweenness_centrality', 'num_second_degree_neighbors']
)


fig = points.properties(width=1600, height=800)
fig.save('figures/synthetic_dataset/synthetic_bipartite_betweenness_vs_cardinality_all_no_text.svg')
fig


In [ ]:
vals_to_select = ['ME', 'MD', 'AL', 'Georgia', 'Ram', 'GT', 'California', 'ES', 'Florida', 'Cuba', 'Jamaica', 'Lincoln', 'Jaguar', 'Elan', 'Conroy', 'Virginia', 'Mace', 'Phoenix']
homograph_df_filtered = homograph_df.loc[homograph_df['node'].isin(vals_to_select)]
homograph_df_filtered

In [ ]:
points = alt.Chart(homograph_df_filtered, title='').mark_circle(size=300, clip=True).encode(
    x=alt.X('num_second_degree_neighbors:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Cardinality', labelFontSize=28, titleFontSize=30)),
    y=alt.Y('betweenness_centrality:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Betweenness Centrality', format=".1e", labelFontSize=28, titleFontSize=30, tickCount=8)),
    tooltip=['node', 'betweenness_centrality', 'num_second_degree_neighbors']
)

text = points.mark_text(
    fontSize=32,
    align='center',
    baseline='middle',
    dy=-25
).encode(
    text='node'
)

fig = (text + points).properties(width=1600, height=800)
fig.save('figures/synthetic_dataset/synthetic_bipartite_betweenness_vs_cardinality.svg')
fig

## Local Clustering Coefficients Evaluation

In [ ]:
# Top-k graph for LCC measure
topk_graph_LCC = alt.Chart(synthetic_bipartite_graph_stats_df.nsmallest(55, 'local_clustering_coefficient'), title='').mark_bar(size=20).encode(
    x=alt.X('node:N', sort='y', axis=alt.Axis(title='', labelAngle=-40, labelFontSize=30)),
    y=alt.Y('local_clustering_coefficient:Q', scale=alt.Scale(domain=(0, 0.55)), axis=alt.Axis(title='Local Clustering Coefficient', labelFontSize=30, titleFontSize=30, tickCount=8)),
    color=alt.Color('is_homograph:N', legend=alt.Legend(title='Value Type', titleLimit=0, labelLimit=500))
).properties(width=2800, height=600)
topk_graph_LCC = topk_graph_LCC.configure_axis(labelLimit=550)

topk_graph_LCC = topk_graph_LCC.configure_legend(
    labelFontSize=38,
    symbolSize=700,
    titleFontSize=35,
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top-left'
)

topk_graph_LCC.save('figures/synthetic_dataset/synthetic_bipartite_LCC_topk_55_wide.svg')
topk_graph_LCC

In [ ]:
# Plot of all the homograph values and their LCC scores
homographs_graph_LCC = alt.Chart(synthetic_bipartite_graph_stats_df[synthetic_bipartite_graph_stats_df['is_homograph'] == 'homograph'], title='').mark_bar(size=24).encode(
    x=alt.X('node:N', sort='y', axis=alt.Axis(title='', labelAngle=-40, labelFontSize=23)),
    y=alt.Y('local_clustering_coefficient:Q', axis=alt.Axis(title='Local Clustering Coefficient', labelFontSize=26, titleFontSize=30, tickCount=8)),
    color=alt.Color('is_homograph:N', legend=alt.Legend(title='Value Type', titleLimit=0, labelLimit=500))
).properties(width=1600, height=800)
homographs_graph_LCC = homographs_graph_LCC.configure_axis(labelLimit=550)

homographs_graph_LCC = homographs_graph_LCC.configure_legend(
    labelFontSize=38,
    symbolSize=700,
    titleFontSize=35,
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top-left'
)

homographs_graph_LCC.save('figures/synthetic_dataset/synthetic_bipartite_LCC_homographs.svg')
homographs_graph_LCC